In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import torch
from torch import nn
from torch.optim import Adam
from transformers import BertModel, BertConfig
from tqdm import tqdm


# Import database
fos_dataframe = pd.read_csv('ROUTE TO DATABASE')

#Import tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased') 

#Create and organise the different labels
labels = {'metaphor':0,
          'metonymy':1,
          'simile':2,
          'neutural':3,
          }
id2label = {
    0:'metaphor',
    1:'metonymy',
    2:'simile',
    3:'neutral'
}

# Create lables for texts
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['type']]
        self.texts = [tokenizer(str(text),
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['word']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y


In [ ]:
#Create classes for different text types
class PredictDataset(torch.utils.data.Dataset):
    def __init__(self, list_of_strings):
        self.texts = [tokenizer(str(st),
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for st in list_of_strings]

    def __len__(self):
        return len(self.texts)

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        return batch_texts

In [ ]:
#Test the length of different dataframes for training
np.random.seed(112)
df_train, df_val, df_test = np.split(fos_dataframe.sample(frac=1, random_state=42), 
                                     [int(.8*len(fos_dataframe)), int(.9*len(fos_dataframe))])

print(len(df_train),len(df_val), len(df_test))

In [ ]:
#Set boundries and configurations for the neural network
config = BertConfig
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()
        self.config = config()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
#Set training parameters
EPOCHS = 5
model = BertClassifier()
LR = 1e-6

#Configure training epochs
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [ ]:
#Train and save the trained model
train(model, df_train, df_val, LR, EPOCHS) 
torch.save(model, 'ROUTE TO SAVE MODEL')